# Notebook: Use NN to predict disease from chemicals using Opa2Vec vectors
<b> Author: </b> Ian Coleman <br>
<b> Purpose: </b> Take the vectors created in the opa2vec notebook. This took chemical go functions
    and disease go function, creating vectors for the chemicals. Train a NN to predict diseases from these chemical
    vectors

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### 1. Import Chem Vectors and Pre-Process them

In [3]:
# Import vec file
with open('AllVectorResults.lst', 'r') as file:
    text = file.read()

In [4]:
# Strip and split it into list of lists [chem, vec]
text = text.replace('\n', '')
text = text.split(']')
text = [item.strip().split(' [') for item in text]

In [5]:
# Turn it into a data frame
df = pd.DataFrame(text)
df.columns = ['ChemicalID', 'Vector']
# df.head()

In [6]:
# Clean
df = df.dropna()
df['Vector'] = df.Vector.map(lambda x: x.rstrip().lstrip().replace('    ', ' ').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))

In [7]:
# Split the vector into a column per number
vec_split = df['Vector'].str.split(',', expand=True)
df = df.join(vec_split, lsuffix='_df', rsuffix='_vec_split')

In [8]:
# df.loc[:,0].head()
# BCE binary classification --> The loss function recommended by Jun
# sigmoid output


### 2. Add Diseases to DF
Binary encode presence of positive association between each disease and each chem

In [17]:
# Import disease list (created in opa2vec notebook that created vectors)
diseases = pd.read_csv('diseases.lst', header=None, skiprows=1) # Skipping first row as will be nan
diseases.shape # 1264 diseases...

In [20]:
df.head()

,ChemicalID,Vector,0,1,2,3,4,5,6,7,...,190,191,192,193,194,195,196,197,198,199
0,D015032,"-0.01185622,-0.31878912,-0.89908963,0.07175528...",-0.01185622,-0.31878912,-0.89908963,0.07175528,0.20856386,-0.6810764,0.7155862,0.3877636,...,0.4799509,0.49532512,-0.40308106,0.440478,-0.14763047,-0.33496988,0.21969438,0.49740723,-0.08506261,0.07757662
1,C085514,"0.0223429,0.1116555,0.02859181,-0.1335976,-0.2...",0.0223429,0.1116555,0.02859181,-0.1335976,-0.23470162,0.034631,0.12097855,0.00488628,...,0.02691469,0.01887334,-0.05138162,0.11950745,-0.02516304,-0.10212526,0.08233052,0.1476164,-0.0013641,0.07410253
2,C104536,"4.91102971e-02,1.35097533e-01,-2.54380330e-03,...",4.91102971e-02,1.35097533e-01,-2.54380330e-03,-1.39362305e-01,-2.44591922e-01,4.78408448e-02,1.43908396e-01,-2.37005409e-02,...,4.86359373e-02,-1.25287324e-02,-3.15188430e-02,1.46263734e-01,-4.74830456e-02,-1.13526262e-01,9.72839370e-02,1.53202027e-01,1.04252342e-02,9.23839062e-02
3,C088658,"-1.5123323e-02,-3.2596567e-01,-1.0544300e+00,2...",-1.5123323e-02,-3.2596567e-01,-1.0544300e+00,2.0313551e-01,3.5050040e-01,-8.2078874e-01,8.6272287e-01,3.4606051e-01,...,4.3986505e-01,4.6400654e-01,-4.5179743e-01,3.4801915e-01,-2.1653381e-01,-2.9522970e-01,2.7476230e-01,5.2579957e-01,2.4951532e-02,8.6974278e-02
4,D014635,"-6.37703110e-03,-4.31791008e-01,-1.22665536e+0...",-6.37703110e-03,-4.31791008e-01,-1.22665536e+00,1.68801114e-01,4.97640580e-01,-9.30020690e-01,8.01523268e-01,5.06105006e-01,...,6.11794293e-01,5.17952442e-01,-3.92865539e-01,5.26886761e-01,-2.12535933e-01,-3.57861817e-01,3.22705418e-01,4.33798850e-01,-1.67800769e-01,7.85325244e-02


In [21]:
diseases.head()

,0
0,C2750090
1,C0342708
2,C0039292
3,C1970456
4,C1845028
